In [ ]:
pip install mediapipe opencv-python numpy


In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import os
import zipfile
import gdown
from google.colab import drive


drive.mount('/content/drive')


mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)


def crop_face(image_path, output_path, size=(224, 224)):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read {image_path}")
        return
    h, w, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(img_rgb)
    if not results.multi_face_landmarks:
        print(f"No face detected in {os.path.basename(image_path)}")
        return

    face_landmarks = results.multi_face_landmarks[0]
    xs = [lm.x * w for lm in face_landmarks.landmark]
    ys = [lm.y * h for lm in face_landmarks.landmark]

    x_min, x_max = int(min(xs)), int(max(xs))
    y_min, y_max = int(min(ys)), int(max(ys))

    # tighten vertically to remove hair
    y_min += int(0.1 * (y_max - y_min))
    y_max -= int(0.05 * (y_max - y_min))

    face_crop = img[y_min:y_max, x_min:x_max]
    if face_crop.size == 0:
        print(f"Invalid crop for {os.path.basename(image_path)}")
        return

    face_resized = cv2.resize(face_crop, size)
    cv2.imwrite(output_path, face_resized)

//we are using drive link to upload raw images and get the output-a folder of output named cropped images is stored in repo

input_folder = "/content/drive/MyDrive/datasets/victims"   
output_folder = "/content/drive/MyDrive/cropped_faces"
zip_path = "/content/drive/MyDrive/cropped_faces.zip"

os.makedirs(output_folder, exist_ok=True)


for file in os.listdir(input_folder):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        input_path = os.path.join(input_folder, file)
        output_path = os.path.join(output_folder, f"cropped_{file}")
        crop_face(input_path, output_path)


with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir(output_folder):
        zipf.write(os.path.join(output_folder, file),
                   arcname=file,
                   compress_type=zipfile.ZIP_DEFLATED)

print(f" All cropped faces zipped successfully at: {zip_path}")
